In [1]:
import torch 
from torch.utils import data 
from torch.autograd import Variable 
import torchvision
from torchvision.datasets import mnist
import matplotlib.pyplot as plt

In [2]:
from datascience import Table
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')
import json
import pandas as pd

In [3]:
training = pd.read_csv('warframe_data.csv')
Train=Table.from_df(training)
Train

time,volume,min_price,max_price,avg_price,median_price,warframe,available,ducats,release,hot,date,weekend
2019/1/1,69,56,80,68,70,Banshee,4,260,0,1,1,0
2019/1/2,74,65,80,72.5,74,Banshee,4,260,0,1,2,0
2019/1/3,61,61,85,73,75,Banshee,4,260,0,1,3,0
2019/1/4,50,61,85,73,75,Banshee,4,260,0,1,4,0
2019/1/5,55,65,80,72.5,73.5,Banshee,4,260,0,1,5,1
2019/1/6,72,60,80,70,70,Banshee,4,260,0,1,6,1
2019/1/7,56,65,85,75,75,Banshee,4,260,0,1,7,0
2019/1/8,59,63,85,74,75,Banshee,4,260,0,1,8,0
2019/1/9,37,72,80,76,79.5,Banshee,4,260,0,1,9,0
2019/1/10,38,62,84,73,75,Banshee,4,260,0,1,10,0


In [175]:
features=[]
for i in range(1335):
    feature1=[]
    feature1.append(Train.row(i)[7])
    feature1.append(Train.row(i)[8]/100)
    feature1.append(Train.row(i)[9])
    feature1.append(Train.row(i)[10])
    feature1.append(Train.row(i)[11]/10)
    feature1.append(Train.row(i)[12])
    features.append(feature1)
features[:5]

[[4, 2.6, 0, 1, 0.1, 0],
 [4, 2.6, 0, 1, 0.2, 0],
 [4, 2.6, 0, 1, 0.3, 0],
 [4, 2.6, 0, 1, 0.4, 0],
 [4, 2.6, 0, 1, 0.5, 1]]

In [176]:
labels=[]
for i in range(1335):
    label=[]
    label.append(Train.row(i)[1])
    label.append(Train.row(i)[2])
    label.append(Train.row(i)[3])
    label.append(Train.row(i)[4])
    labels.append(label)
labels[:5]

[[69, 56, 80, 68.0],
 [74, 65, 80, 72.5],
 [61, 61, 85, 73.0],
 [50, 61, 85, 73.0],
 [55, 65, 80, 72.5]]

In [177]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [213]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(6,24) 
        self.fc2 = nn.Linear(24,12)
        self.fc3 = nn.Linear(12,4)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [214]:
class MyDataset(Dataset):
    def __init__(self,labels,features):
        super(MyDataset,self).__init__()
        self.labels=labels
        self.features=features
    def __len__(self):
        return self.features.shape[0]
    def __getitem__(self,idx):
        feature=self.features[idx]
        label=self.labels[idx]
        return {'feature':feature, 'label':label}

In [215]:
def train_epoch(loader):
    total_loss=0.0
    for i,data in enumerate(loader):
        featurest=data['feature'].float()
        labelst=data['label'].float()
        optimizer.zero_grad()
        out=mnet(featurest)
        loss=loss_fn(out,labelst)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print ('loss', total_loss/i)

In [239]:
mnet=Net()
loss_fn=torch.nn.MSELoss()
optimizer=torch.optim.SGD(mnet.parameters(),lr=0.0001,momentum=0.01,weight_decay=1e-8)

dataset=MyDataset(np.asarray(labels),np.asarray(features))
load=DataLoader(dataset,shuffle=True,batch_size=100)

for epoch in range(1000):
    train_epoch(load)

loss 31054.17578125
loss 30537.702674278848
loss 21208.9208984375
loss 10199.054612379809
loss 6617.003079927885
loss 4120.304912860577
loss 3067.138653094952
loss 2708.5537391075723
loss 2525.2543006310098
loss 2457.553448016827
loss 2404.5890080378604
loss 2218.53364445613
loss 2143.40678523137
loss 2230.5588660606973
loss 2167.870896559495
loss 1946.7977294921875
loss 1842.2081298828125
loss 1748.3250262920674
loss 1704.852276141827
loss 1616.122333233173
loss 1604.9166212815505
loss 1819.0872802734375
loss 1508.8524592472957
loss 1451.6295166015625
loss 1556.8845872145432
loss 1447.350614107572
loss 1356.1800255408655
loss 1350.192636343149
loss 1396.979243351863
loss 1507.4406034029448
loss 1525.9587777944712
loss 1418.9405141977163
loss 1250.4829852764424
loss 1379.0920879657451
loss 1195.33396089994
loss 1315.796602689303
loss 1442.8056077223557
loss 1247.187265249399
loss 1391.2123647836538
loss 1211.9847928560698
loss 1436.391869178185
loss 1129.5049954927886
loss 1412.8183640

loss 489.9691913311298
loss 534.7378375713641
loss 611.5460298978365
loss 702.2334266075721
loss 489.16659076397235
loss 458.5401822603666
loss 532.716306246244
loss 576.8397803673378
loss 475.7132897010216
loss 477.50958251953125
loss 748.3490811861478
loss 962.9896662785457
loss 474.68602341871997
loss 491.49415705754205
loss 478.80223435621997
loss 443.77784846379205
loss 463.19942180926984
loss 545.8618985689603
loss 515.2912785456731
loss 642.8951650766226
loss 547.0884775015024
loss 1012.0518024151141
loss 514.9357346754807
loss 449.31688044621393
loss 529.9271193284255
loss 442.87352576622595
loss 489.1936927208534
loss 500.62445772611176
loss 517.4081655649038
loss 496.943354679988
loss 506.7808603140024
loss 450.74425330528845
loss 466.20184560922473
loss 524.6943805401141
loss 444.62240835336536
loss 517.7755432128906
loss 507.04322697566107
loss 730.4423076923077
loss 485.9379319411058
loss 537.6105323204628
loss 461.23295241135816
loss 552.9393122746394
loss 537.52096792367

loss 387.4561298076923
loss 392.1962679349459
loss 383.00682537372296
loss 548.3392498309796
loss 471.15578988882214
loss 380.49549278846155
loss 374.07523052509015
loss 444.78927142803485
loss 528.5377338115985
loss 442.6417752779447
loss 417.97601905235877
loss 383.2708082932692
loss 430.4979717548077
loss 484.7690218411959
loss 690.4173184908353
loss 440.29962158203125
loss 500.0365060659555
loss 380.2584181565505
loss 380.1591033935547
loss 419.3743966909555
loss 453.0593003493089
loss 392.75997455303485
loss 396.01510854867786
loss 417.06943570650543
loss 382.8860180194561
loss 515.9309316781851
loss 384.09537212665265
loss 452.14776611328125
loss 715.1645343487079
loss 387.5835453913762
loss 432.35707209660455
loss 358.30057232196515
loss 379.9879197340745
loss 389.9098299466647
loss 398.1811722975511
loss 749.7041637714093
loss 384.71246337890625
loss 387.9022874098558
loss 387.21100088266223
loss 453.20306631234973
loss 382.8333212045523
loss 381.85847825270434
loss 374.2141019

In [249]:
k=562
state=features[k]
innput=Variable(torch.FloatTensor(state))
output=mnet(innput).detach().numpy()
print(output)
print(labels[k])
Train.row(k)

[171.18152 133.69408 160.19861 146.79666]
[169, 138, 164, 151.0]


Row(time='2019/1/29', volume=169, min_price=138, max_price=164, avg_price=151.0, median_price=150.0, warframe='Mesa', available =3, ducats=260, release=1, hot=3, date=29, weekend=0)

In [250]:
torch.save(mnet.state_dict(),'DNNmodel.pth')